In [ ]:
pip install glob --upgrade
pip install os --upgrade
pip install numpy --upgrade
pip install pandas --upgrade
pip install time --upgrade
pip install matplotlib --upgrade

pip install cv2 --upgrade

pip install torch --upgrade


In [3]:
#pip install torchstat
!pip install torchsummary

ERROR: Could not find a version that satisfies the requirement torchsummary (from versions: none)
ERROR: No matching distribution found for torchsummary


In [1]:
import glob
import os
import numpy # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import matplotlib
import matplotlib.pyplot
import cv2
import random
import sklearn
import sklearn.model_selection

import torch
#import torchvision
import torchsummary 
import torchmetrics

ModuleNotFoundError: No module named 'torchsummary'

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data

In [ ]:
class customDataset(torch.utils.data.Dataset):
    
    def __init__(self, data = None, data_test = None, test_factor = 0.2, for_training = True,
                 data_type = None,
                 transform = None, multiple_processes = 2, seed = None):
        
        
        self.problem = "segmentation"
        self.problem_type = "supervised"
        
                
        self.data_loaded = False
        
        self.parameter_check(data = data, data_test = data_test, test_factor = test_factor,
                             for_training = for_training,
                             data_type = data_type, multiple_processes = multiple_processes, seed = seed)
        
        
        
        
        try:
            self.transform = transform
        except NameError:
            self.transform = None
        
      
    def parameter_check(self, data, data_test, test_factor, for_training, data_type, multiple_processes, seed):
                   
        data_type.lower()
        if (data_type != "alphanumeric" or data_type != "image" or
            data_type != "audio" or data_type != "video" or
            data_type != "mix"):
            
            
            #TODO raise error 
            pass
        
        self.data_type = data_type
        
        
        
        self.multiple_processes = multiple_processes
        self.for_training = for_training
        
        
        if (data != None):
            self.generate_dataset(data, data_type)
        
        if (data_test is None):
            self.train_test_split(test_factor = test_factor, seed = seed)
        
        else:
            self.data_train = self.data
            self.data_test = data_test
            
            self.train_size = len(self.data_train)
            self.test_size = len(self.data_test)
            self.test_factor = len(self.data_test)/len(self.data)
            
            
    def initiate_process(self, multiple_processes, batch_size, shuffle, drop_last):
        
        self.preprocessing()
        
        self.init_dataloader(for_training = self.for_training, batch_size = batch_size, multiple_processes = multiple_processes,
                             shuffle = shuffle, drop_last = drop_last)
        
        
            
    def generate_dataset (self, data, data_type):
        
        """
        Data can be a dataset, data_directory, a Database, 
        """
        start_time = time.process_time()
        
        if os.path.isdir(data):
             
            
            if (data_type == "alphanumeric"):
                pass
        
            elif (data_type == "image" and self.problem_type == "supervised"):
                
                # Must have image data and class (Y) data
                
                
                if (self.problem == "classification"):
                    pass
                elif (self.problem == "detection"):
                    pass
                elif (self.problem == "segmentation"):
                    
                    # must have Binary classes (Mask of Yes Data and Mask of No Data)
                    # define directory paths
                    data_directory = data
                    
                    classes = list(map(os.path.basename,[f.path for f in os.scandir(data_directory) if f.is_dir()]))
    
                    image_folders = ["images", "imgs", "image", "Images", "Imgs", "Image"]
                    mask_folders = ["masks", "mks", "mask", "Masks", "Mks", "Mask"]
            
                     # initialize empty lists for image and mask paths
                    image_paths = []
                    mask_paths = []
                    
                    # iterate through image directories and add image paths to list
                    for classe in classes:
                        classe_directory = os.path.join(data_directory, classe)
                        
                        for image_folder in image_folders:
                            path = os.path.join(classe_directory, image_folder)
                            
                            if os.path.exists(path):
                                                               
                                image_paths.extend(glob.glob(path + '/*.[jJ][pP][gG]', recursive=False))
                                image_paths.extend(glob.glob(path + '/*.[jJ][pP][eE][gG]', recursive=False))
                                image_paths.extend(glob.glob(path +'/*.png', recursive=False))
                                image_paths.extend(glob.glob(path + '/*.webp', recursive=False))

                        # iterate through mask directories and add mask paths to list
                        for mask_folder in mask_folders:
                            path = os.path.join(classe_directory, mask_folder)
                            if os.path.exists(path):
                                mask_paths.extend(glob.glob(path + '/*.[jJ][pP][gG]', recursive=False))
                                mask_paths.extend(glob.glob(path + '/*.[jJ][pP][eE][gG]', recursive=False))
                                mask_paths.extend(glob.glob(path +'/*.png', recursive=False))
                                mask_paths.extend(glob.glob(path + '/*.webp', recursive=False))

                    # create dataframe from image and mask paths
                    data = {'X': image_paths, 'Y': mask_paths}
                    dataset = pandas.DataFrame(data)
                    
                elif (self.problem == "mix"):
                    pass
        
            elif (data_type == "audio"):
                pass
            elif (data_type == "video"):
                pass
            elif (data_type == "mix"):
                pass
            
            
            self.data = dataset
            
            end_time = time.process_time()

            self.data_loading_time = end_time - start_time
            
            print("Data Loading Time: {0} minutes, {1:.3f} seconds".format(int(self.data_loading_time/60), self.data_loading_time%60))
            
        else:
            # TODO check if dataset
            self.data = data
            #TODO check if connection
        
        self.data_loaded = True
        
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        if (self.data_type == "image" and self.problem == "segmentation" and
            self.problem_type == "supervised"):
            
            X = self.data.loc[index]["X"]
            Y = self.data.loc[index]["Y"]
            
            X = numpy.array(Image.open(X_directory).convert("RGB"))
            Y = numpy.array(Image.open(Y_directory).convert("L"), dtype=np.float32)
            Y[Y == 255.0] = 1.0
            
            if self.transform is not None:
                augmentations = self.transform(image=image, mask=mask)
                image = augmentations["image"]
                mask = augmentations["mask"]
            
            img_path = os.path.join(self.image_dir, self.images[index])
            mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        

        

        return X, Y
    
    
    def get_random_sample (self, sample_number = 2, img_show = False, img_transform = False):
        
        if (self.data_type == "image" and self.problem == "segmentation" and
            self.problem_type == "supervised"):
            
            
            samples_index = numpy.random.randint(self.__len__(), size = sample_number)
            
            print(samples_index)
            
            X_directory = self.data.loc[samples_index]["X"]
            Y_directory = self.data.loc[samples_index]["Y"]
        
        
            if (img_transform == True):
                pass
        
            if (img_show == True):
                
                fig, axes = matplotlib.pyplot.subplots(nrows=sample_number, ncols=2, figsize = (8,5*sample_number))
                
                
                for i in range(0, len(samples_index), 1):
                    
                    
                    X_img = cv2.imread(X_directory[samples_index[i]])
                    Y_img = cv2.imread(Y_directory[samples_index[i]])
                
                    axes[i][0].imshow(X_img)
                    axes[i][1].imshow(Y_img)
                    
                    axes[i][0].set_title(X_directory[samples_index[i]], fontsize=8)
                    axes[i][1].set_title(Y_directory[samples_index[i]], fontsize=8)
                
                matplotlib.pyplot.show()
        
        else:
            pass
        
        return X_directory, Y_directory
    
    def analize_sample (self, index = None, sample = None, with_model = False):
        
        image = cv2.imread(image_path)
        height, width, channels = image.shape
        laplacian = numpy.array(cv2.Laplacian(image, cv2.CV_64F))
        
    def evaluate_data (self):
        
        start_time = time.process_time()
        
        if (self.data_loaded == False):
            #TODO raise error
            return
        
        if (self.data_type == "image"):
            
            keys = ["height", "width", "channels", "quality"]
            keys_special = ["histogram", ""]
            
            
            def img_info(image_path):
                
                image = cv2.imread(image_path)
                height, width, channels = image.shape
                
                hist_blue = cv2.calcHist([image], [0], None, [256], [0, 256])
                hist_green = cv2.calcHist([image], [1], None, [256], [0, 256])
                hist_red = cv2.calcHist([image], [2], None, [256], [0, 256])
                quality = 1
                
                data = {
                    "height":height,
                    "width":width,
                    "channels":channels,
                    "histogram":[hist_red, hist_green, hist_blue],
                    "quality":quality
                }
                return data
            
            
            if (self.problem_type == "supervised"):
                
                X_info = numpy.array(self.data["X"].apply(img_info))
                Y_info = numpy.array(self.data["Y"].apply(img_info))
            
                X_features = []
                Y_features = []
            
                for key in keys:


                    X_features.append([len([data.get(key) for data in X_info]),
                                       numpy.max([data.get(key) for data in X_info]),
                                       numpy.min([data.get(key) for data in X_info]),
                                       numpy.mean([data.get(key) for data in X_info]),
                                       numpy.median([data.get(key) for data in X_info]),
                                       numpy.std([data.get(key) for data in X_info]),
                                       numpy.unique([data.get(key) for data in X_info])])


                    Y_features.append([len([data.get(key) for data in Y_info]),
                                       numpy.max([data.get(key) for data in Y_info]),
                                       numpy.min([data.get(key) for data in Y_info]),
                                       numpy.mean([data.get(key) for data in Y_info]),
                                       numpy.median([data.get(key) for data in Y_info]),
                                       numpy.std([data.get(key) for data in Y_info]),
                                       numpy.unique([data.get(key) for data in Y_info])])



                fig, axes = matplotlib.pyplot.subplots(2,2, figsize = (10,8))



                X_histogram_average = numpy.mean([data.get("histogram") for data in X_info], axis = 0)            
                Y_histogram_average = numpy.mean([data.get("histogram") for data in Y_info], axis = 0)
                
                axes[0][0].plot(X_histogram_average[0], color = "red")
                axes[0][0].plot(X_histogram_average[1], color = "green")
                axes[0][0].plot(X_histogram_average[2], color = "blue")
                
                axes[0][0].set_title("Average RGB Histogram X")
                
                axes[0][1].plot(Y_histogram_average[0], color = "red")
                axes[0][1].plot(Y_histogram_average[1], color = "green")
                axes[0][1].plot(Y_histogram_average[2], color = "blue")
                
                axes[0][1].set_title("Average RGB Histogram Y")
                
                
                X_img = cv2.imread(self.data.loc[random.randint(0, self.__len__())]["X"])
                # Convert to grayscale
                gray = cv2.cvtColor(X_img, cv2.COLOR_BGR2GRAY)                
                laplacian = cv2.Laplacian(gray, cv2.CV_64F)
                
                axes[1][0].imshow(laplacian, cmap = "gray")
                axes[1][0].set_title("Laplacian")


                dataset_X = pandas.DataFrame(X_features, 
                                           columns = ["count", "max","min","average","median","std", "unique"],
                                           index = keys)

                dataset_Y = pandas.DataFrame(X_features, 
                                           columns = ["count", "max","min","average","median","std", "unique"],
                                           index = keys)

                delta = time.process_time() - start_time
                print("Data Evaluation Time: {0} hours, {1} minutes, {1:.3f} seconds".format(int(delta/24), int(delta/60), delta%60))


                matplotlib.pyplot.show()
                
                return dataset_X, dataset_Y
            
    def preprocessing (self):
        
        print("Preprocessing Process Initiated")
        
        def data_augementation (self):
            pass
        def data_sythesis(self):
            pass
        print("Preprocessing Process Completed")
        pass
   
    def train_test_split(self, test_factor, seed):
        
        self.test_factor = test_factor
        
        if (seed is None):
                X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(
                    self.data["X"],
                    self.data["Y"],
                    test_size = test_factor)
                
        else:
                X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(
                    self.data["X"],
                    self.data["Y"],
                    test_size = test_factor,
                    random_state = seed)
            
        
        self.data_train = pandas.concat([X_train, Y_train], axis = 1)
        self.data_test = pandas.concat([X_test, Y_test], axis = 1)
            
        self.train_size = len(self.data_train)
        self.test_size = len(self.data_test)

        
        fig, axes = matplotlib.pyplot.subplots(1,1, figsize = (3,3))
        plot = axes.bar(["Train", "Test"], [self.train_size, self.test_size])
        axes.set_title("Train Test Split Factor: {}".format(self.test_factor))
        axes.set_xlabel("Datasets")
        axes.set_ylabel("Sample Count")
        
        '''
        for rect, label in plot:
            height = rect.get_height()
            top = height
            ax.text(rect.get_x() + rect.get_width() / 2, height + 5, top , ha="center", va="bottom")
        '''
        #axes.set_grid()
        #axes.legend()
        matplotlib.pyplot.show()
            
            
    def init_dataloader(self, for_training = True, batch_size = 32, multiple_processes = 2, shuffle = True, drop_last = False):           

        if (for_training == True):
            
            class Auxiliar(torch.utils.data.Dataset):
                def __init__(self, data):
                    self.data = data
        
                def __getitem__(self, index):
                    x, y = self.data[index]
                    return (x, y)
    
                def __len__(self):
                    return len(self.data)
            
            train = Auxiliar(self.data_train)
            test = Auxiliar(self.data_test)

            self.Dataloader_train = torch.utils.data.DataLoader(train,
                                                  batch_size = batch_size,
                                                  num_workers = multiple_processes,
                                                  shuffle = shuffle,
                                                  drop_last = drop_last)

            self.Dataloader_test = torch.utils.data.DataLoader(test,
                                                  batch_size = batch_size,
                                                  num_workers = multiple_processes,
                                                  shuffle = shuffle,
                                                  drop_last = drop_last)


        elif (for_training == False):

          #Not for training. For predicting
          self.Dataloader = torch.utils.data.DataLoader(self.dataset,
                                                  batch_size = 1,
                                                  num_workers = multiple_processes,
                                                  shuffle = shuffle,
                                                  drop_last = drop_last)

        print("DataLoader Process Complete")
 

In [ ]:
data_directory = r"../input/crack-dataset/Concrete"
data = customDataset(data = data_directory, test_factor = 0.2, data_type = "image",
                 transform = None, multiple_processes = 2)

In [ ]:
data.get_random_sample(5, img_show = True)

In [ ]:
data.evaluate_data()

In [ ]:
data.initiate_process(multiple_processes = 1, batch_size = 32, shuffle = True, drop_last = False)

# Model

In [ ]:
import torch
import torch.nn
import torch.nn.functional
import torchvision
import torchvision.transforms



class Layer ():

    def __init__(self, type, activationFunction) -> None:
        super().__init__()
    
        self.layer = type
        self.activationFunction = activationFunction

    def forward (self, X):

        if self.activationFunction is not None:
            return self.activationFunction(self.layer(X))
        
        return self.layer(X)



class DoubleConvolutionBlock ():

    def __init__(self, channels_input, channels_output:int, mode, final_output = 1) -> None:
        super().__init__()

        self.channels_input = channels_input
        self.channels_output = channels_output
        if (mode.lower() == "final"):
            self.final_output = final_output

        self.layers = []

        self.build(mode)

    
    def build (self, mode):

        layer1 = torch.nn.Conv2d(in_channels = self.channels_input, 
        out_channels = self.channels_output, kernel_size = (3,3),
                                stride = 1, padding = 1, dilation = 1, bias = True,
                                padding_mode = "zeros", device=None)

        activation1 = torch.nn.functional.relu



        layer2 = torch.nn.Conv2d(in_channels = self.channels_output, 
        out_channels = self.channels_output, kernel_size = (3,3),
                                stride = 1, padding = 1, dilation = 1, bias = True,
                                padding_mode = "zeros", device=None)
        activation2 = torch.nn.functional.relu


        if (mode == "Down"):
            #layer3 = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2, padding=0, dilation=1)
            #activation3 = None
            self.layers.append(Layer(layer1, activation1))
            self.layers.append(Layer(layer2, activation2))

        elif (mode == "Up"):
            layer3 = torch.nn.ConvTranspose2d(self.channels_output, (self.channels_output)//2,
             kernel_size = (2, 2), stride=2, padding=0, dilation=1)
            activation3 = None
            
            self.layers.append(Layer(layer1, activation1))
            self.layers.append(Layer(layer2, activation2))
            self.layers.append(Layer(layer3, activation3))

        elif (mode == "Final"):

            layer3 = torch.nn.Conv2d(in_channels = self.channels_output, out_channels = self.final_output,
             kernel_size = (1,1),stride = 1, padding = 0, dilation = 1, bias = True,
                                padding_mode = "zeros", device=None)            
            activation3 = None

            self.layers.append(Layer(layer1, activation1))
            self.layers.append(Layer(layer2, activation2))
            self.layers.append(Layer(layer3, activation3))

        
        

    def forward (self, X):

        for layer in self.layers:

            X = layer.forward(X)
        
        return X



class Model ():

    def __init__(self, channels_input = 3, channel_output = 3, img_size = (255, 255)) -> None:
        

        self.channels_input = channels_input
        self.channel_output = channel_output

        self.layers = []

        self.built = False
        self.trained = False
        
        
        self.problem = "segmentation"
        self.problem_type = "supervised"
                

    def build(self):

        
        # Encoder Chain
        # The contracting path follows the typical architecture of a convolutional network.
        # It consists of the repeated application of two 3×3 convolutions
        # (unpadded convolutions), each followed by a rectified linear unit (ReLU)
        # and a 2×2 max pooling operation with stride 2 for downsampling.
        # At each downsampling step we double the number of feature channels.

        self.block1 = DoubleConvolutionBlock(self.channels_input, 64, "Down")
        self.block2 = DoubleConvolutionBlock(64, 128, "Down")
        self.block3 = DoubleConvolutionBlock(128, 256, "Down")
        self.block4 = DoubleConvolutionBlock(256, 512, "Down")

    
        # Decoder Chain
        # Every step in the expansive path consists of an upsampling of the feature map
        # followed by a 2×2 convolution (“up-convolution”) that halves the number
        # of feature channels, a concatenation with the correspondingly cropped feature map
        # from the contracting path, and two 3×3 convolutions, each followed by a ReLU

        self.block5 = DoubleConvolutionBlock(512, 1024, "Up")
        
        self.block6 = DoubleConvolutionBlock(1024, 512, "Up")
        self.block7 = DoubleConvolutionBlock(512, 256, "Up")
        self.block8 = DoubleConvolutionBlock(256, 128, "Up")

        self.block9 = DoubleConvolutionBlock(128, 64, "Final", self.channel_output)
    

        self.built = True

    def forward (self, X):

        self.connections = []

        X = self.block1.forward(X)
        self.connections.append(X)
        X = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2, padding=0, dilation=1)(X)

        X = self.block2.forward(X)
        self.connections.append(X)
        X = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2, padding=0, dilation=1)(X)

        X = self.block3.forward(X)
        self.connections.append(X)
        X = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2, padding=0, dilation=1)(X)

        X = self.block4.forward(X)
        self.connections.append(X)
        X = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2, padding=0, dilation=1)(X)




        X = self.block5.forward(X)
        
        
        if (X.shape != self.connections[-1].shape):
            X = torchvision.transforms.functional.resize(X, size = self.connections[-1].shape[2:])
        X = torch.cat((self.connections[-1], X), dim = 1)
        X = self.block6.forward(X)

        if (X.shape != self.connections[-2].shape):
            X = torchvision.transforms.functional.resize(X, size = self.connections[-2].shape[2:])
        X = torch.cat((self.connections[-2], X), dim = 1)
        X = self.block7.forward(X)
        
        if (X.shape != self.connections[-3].shape):
            X = torchvision.transforms.functional.resize(X, size = self.connections[-3].shape[2:])
        X = torch.cat((self.connections[-3], X), dim = 1)
        X = self.block8.forward(X)

        if (X.shape != self.connections[-4].shape):
            X = torchvision.transforms.functional.resize(X, size = self.connections[-4].shape[2:])
        X = torch.cat((self.connections[-4], X), dim = 1)
        
        
        return self.block9.forward(X)


    def build_UNet_modified(self) -> None:

        #TODO


        layer1 = torch.nn.Conv2d(in_channels = 3, out_channels = 127, kernel_size = (3,3),
                                stride = 1, padding = 1, dilation = 1, bias = False,
                                padding_mode = "same", device=None)
        activation1 = None

        layer2 = torch.nn.BatchNorm2d(32)
        activation2 = torch.nn.functional.relu


        layer3 = torch.nn.Conv2d(in_channels = 3, out_channels = 127, kernel_size = (3,3),
                                stride = 1, padding = 1, dilation = 1, bias = False,
                                padding_mode = "same", device=None)
        activation3 = None

        
        layer4 = torch.nn.BatchNorm2d(32)
        activation4 = torch.nn.functional.relu



        layer5 = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2)
        activation5 = None

        
        self.layers.append(Layer(layer1, activation1))
        self.layers.append(Layer(layer2, activation2))
        self.layers.append(Layer(layer3, activation3))
        self.layers.append(Layer(layer4, activation4))
        self.layers.append(Layer(layer5, activation5))

        #Down Block
    




        self.built = True


    def debbugFoward (self):

        batch_size = 5
        img_size = 160
        X = torch.randn((batch_size, self.channels_input, img_size, img_size))

        if (self.built == False):
            self.build()

        pred = self.forward(X)
        print("Input Shape: ", X.shape)
        print("Predict Shape: ", pred.shape)
        
        condition = (batch_size == pred.shape[0]) and (pred.shape[1] == self.channel_output)

        assert condition, AssertionError

        return True

    def summary(self):
        
        if(self.problem == "segmentation"):
            torchsummary.summary(self, (self.channels_input, 200, 200))
   
    def loadModel (self, path):
        self.load_state_dict(torch.load(path))
        
    def saveModel(self, path):
        torch.save(self.state_dict(), path)
        
    def eval(self):
        pass
    
    

m = Model(channels_input = 3, channel_output = 1)

m.debbugFoward()


# Project

In [ ]:
class Project():
    
    def __init__ (self, model, dataset, problem, problem_type,
                   ):  
        
        
        self.parameter_check(problem, problem_type)
        
        self.model = model
        self.dataset = dataset
        
        self.dataset.problem = self.problem
        self.dataset.problem_type = self.problem_type
        
        
        
        self.savePath = None
        
        
    def parameter_check(problem, problem_type):
        
        problem = problem.lower()
        if (problem != "regression" or problem != "classification" or
            problem != "detection" or problem != "segmentation" or
            problem != "anomaly" or problem != "mix"):
            
            #TODO
            pass
        
        self.problem = problem
        
        problem_type = problem_type.lower()
        if (problem_type != "supervised" or problem_type != "unsupervised" or
            problem_type != "reinforcement" or problem_type != "semi-supervised" or
            problem_type != "transfer" or problem_type != "active" or 
            problem_type != "generative" or problem_type != "recommendation"):
            
            #TODO
            pass
        
        self.problem_type = problem_type
        
        
        
    def problem_summary(self):
        
        
    def train_batch(self, data):
        
        if (self.problem_type == "supervised" and
            self.problem_type == "classification"):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            inputs = inputs.to(self.device)
            labels = labels.to(self.device)
        
            # track history if only in train
            with torch.set_grad_enabled(True):
                
                # 1. Forward pass
                # forward + backward + optimize
                if(self.model.baseModel is None):
                    y_pred = self.model.forward(inputs)
                else:
                    y_pred = self.model.baseModel(inputs)
                    
                # 2. Calculate  and accumulate loss
                loss = self.function_loss(y_pred, labels)
                
                # 3. Optimizer zero grad
                self.optimizer.zero_grad()
                
                # 4. Loss backward
                loss.backward()
                
                # 5. Optimizer step
                self.optimizer.step()
            # Calculate and accumulate accuracy metric across all batches
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            
            #y_pred_class = torch.argmax(y_pred, dim=1)
            acc = (y_pred_class == labels).sum().item()/len(y_pred)
        
        return loss.item(), acc
    
    
    
    def train_fold(self, n_epoch, train_loader, best_batch_show = True):
        
        #Epochs
        for epoch in range (0, n_epoch, 1):

            epoch_start = time.time()

            train_loss = []
            train_acc = []
            best_batch_acc = 0.0
            best_batch_loss = 999
            best_batch_iteration = 0

            data_10 = 0
            i = 1

            #train iteration
            if (best_batch_show is True):
                print("Epoch {0}".format(epoch+1), end = ": ")


            for batch_i, data in enumerate(train_loader):

                loss, acc = self.train_batch(data)
                data_10 += len(data)

                # print statistics
                if (best_batch_show is not True):
                    print("Epoch:{0} Batch:{1} Loss = {2:0.4f} Accuracy = {3:0.4f}".format(epoch+1, batch_i+1, loss, acc))

                elif (best_batch_show is True):

                    if (best_batch_acc < acc and best_batch_loss > loss):
                        best_batch_acc = acc
                        best_batch_loss = loss
                        best_batch_iteration = batch_i

                    if (data_10 > self.dataset.trainSize/10 * i):
                        print("{0}-".format(i), end = "")
                        i += 1

                self.train_scores.append([epoch+1, data_10, loss, acc])

                train_loss.append(loss)
                train_acc.append(acc)
    
    
    def trainModel (self, batch_size = 32, n_epochs = 5, KFolds = 0,
                    function_loss = None, optimizer = None,
                    learning_rate = None, device = None, multiple_processes = 2,
                    best_batch_show = True, savePath = None, debbugMode = False,
                    debbugFile_Path = None):
        
        self.nEpoch = n_epochs
        self.KFolds = KFolds

        
                
        self.train_scores = []            #scores of model as number sample increase
        self.train_scores_epochs = []     #scores of model as epoch increase
        self.cv_scores_epochs = []  

        train_start = time.process_time()
        

        print("************************************************************")
        print("Model Training on: {0}".format(self.device))

        if (self.KFolds > 0):
            
            cv_splits = sklearn.model_selection.KFold(n_splits = KFolds, shuffle=True)
            
            for fold, (train_index, cv_index) in enumerate(cv_splits.split(numpy.arange(len(self.dataset.data_train)))):
                
                print("Fold: {0}/{1}".format(fold, self.KFolds),sep = "\n")
                
                train_sampler = torch.utils.dataSubsetRandomSampler(train_index)
                cv_sampler = torch.utils.dataSubsetRandomSampler(cv_index)
                
                train_loader = torch.utils.data.DataLoader(self.dataset.dataset_train, batch_size = self.dataset.batch_size, sampler = train_sampler, drop_last = False)
                cv_loader = torch.utils.data.DataLoader(self.dataset.dataset_train, batch_size = self.dataset.batch_size, sampler = cv_sampler, drop_last = False)
                
                
                self.train_fold(n_epoch = n_epochs, train_loader = train_loader, best_batch_show = best_batch_show)
                
                #Validation iteration

                if (best_batch_show is True):
                    print("Epoch {0}".format(epoch+1), end = ": ")
                
                for batch_i, data in enumerate(cv_loader):
                    loss, acc = self.train_batch(data)
                    
                    cv_loss.append(loss)
                    cv_acc.append(acc)

             
                if (best_batch_show is True):
                    print("\nBest_Batch:{0} Best_Batch_Loss = {1:0.4f} Best_Batch_Accuracy = {2:0.4f}".format(best_batch_iteration+1, best_batch_loss, best_batch_acc))
            
                # Adjust metrics to get average loss and accuracy per batch 
                train_loss_avg = sum(train_loss) / len(self.dataset.Dataloader_train)
                train_acc_avg = sum(train_acc) / len(self.dataset.Dataloader_train)

                self.train_scores_epochs.append([epoch+1, train_acc, train_loss, float((time.time() - epoch_start) // 60), (time.time() - epoch_start) % 60])

                # print statistics
                print("Epoch:{0} Loss_Avg = {1:.4f} Accuracy_Avg = {2:.4f} Time Lapsed = {3}min {4:.2f}seg\n".format(
                    epoch+1, train_loss_avg, train_acc_avg, (time.time() - epoch_start) // 60, (time.time() - epoch_start) % 60))
        

        
        elif (self.KFolds <= 0):
            #no cv
            ###################################################

            self.train_fold(n_epoch = n_epochs, train_loader = train_loader, best_batch_show = best_batch_show)

            if (best_batch_show is True):
                print("\nBest_Batch:{0} Best_Batch_Loss = {1:0.4f} Best_Batch_Accuracy = {2:0.4f}".format(best_batch_iteration+1, best_batch_loss, best_batch_acc))

            # Adjust metrics to get average loss and accuracy per batch 
            train_loss_avg = sum(train_loss) / len(self.dataset.Dataloader_train)
            train_acc_avg = sum(train_acc) / len(self.dataset.Dataloader_train)

            self.train_scores_epochs.append([epoch+1, train_acc, train_loss, float((time.time() - epoch_start) // 60), (time.time() - epoch_start) % 60])


            # print statistics
            print("Epoch:{0} Loss_Avg = {1:.4f} Accuracy_Avg = {2:.4f} Time Lapsed = {3}min {4:.2f}seg\n".format(
                    epoch+1, train_loss_avg, train_acc_avg, (time.time() - epoch_start) // 60, (time.time() - epoch_start) % 60))


            
            
            
        delta_training = time.process_time() - train_start
        print("Model Training Time: {0} hours, {1} minutes, {2:.3f} seconds, {} seconds".format(
                int(delta_training/24),int(delta_training/60), delta_training%60)) 
        print("************************************************************")
    
    
        
        self.model.trained = True
        if (savePath is not None):
            self.savePath = savePath
            self.model.saveModel(savePath)


        if (debbugMode is True and debbugFile_Path is not None):
            file_object = open(debbugFile_Path, 'w')
            for data in self.train_scores_epochs:
                file_object.write(str(data[0]))
                file_object.write('|')
                file_object.write(str(data[1]))
                file_object.write('|')
                file_object.write(str(data[2]))
                file_object.write('|')
                file_object.write(str(data[3]))
                file_object.write('|')
                file_object.write(str(data[4]))

                file_object.write("\n")
            #Close the file
            file_object.close()



        #self.plot_learning_curve()
    
    

In [ ]:
uNet = Model()
uNet.__summary__()

# Model